*   Damian Jurkiewicz - jurkiewicz@student.agh.edu.pl
*   Daniel Klarenbach - klarenbach@student.agh.edu.pl

# Zadanie 1 - Preprocessing

In [77]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [78]:
SZ=28
affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR

def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

In [79]:
def showOpencvImage(image, isGray=False):
    fig = plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap = 'gray')
    plt.show()

In [80]:
def openCVHOG(im):
    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)
    descriptor = np.ravel(hog.compute(im))
    
    return descriptor

In [81]:
from sklearn.metrics import classification_report, confusion_matrix

def print_report(test_y,pred_y):
    mask = pred_y==test_y
    correct = np.count_nonzero(mask)
    print(correct*100.0/pred_y.size)

    cm = confusion_matrix(test_y, pred_y)
    print(cm)
    print(classification_report(test_y, pred_y)) 

In [82]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [83]:
# Zmniejszony zbiór testowy - finalnie pominąć 

train_images = train_images[0:600]
train_labels = train_labels[0:600]
test_images = test_images[0:100]
test_labels = test_labels[0:100]

In [84]:
train_raw = train_images.reshape(len(train_images), 28 * 28)
test_raw = test_images.reshape(len(test_images), 28 * 28)

Przygotowanie zbiorów

In [85]:
train_deskewed =  np.float32([deskew(im) for im in train_raw])
test_deskewed =  np.float32([deskew(im) for im in test_raw])

train_deskewed = np.asarray(train_deskewed).reshape(-1,28*28)
test_deskewed = np.asarray(test_deskewed).reshape(-1,28*28)

# train_deskewed
# test_deskewed

In [86]:
hogdata_train = [openCVHOG(im) for im in train_images]
hogdata_train = np.float32(hogdata_train).reshape(-1,81)

hogdata_test = [openCVHOG(im) for im in test_images]
hogdata_test = np.float32(hogdata_test).reshape(-1,81)

# hogdata_train
# hogdata_test

In [87]:
deskewed_hogdata_train = [openCVHOG(deskew(im)) for im in train_images]
deskewed_hogdata_train = np.float32(deskewed_hogdata_train).reshape(-1,81)

deskewed_hogdata_test = [openCVHOG(deskew(im)) for im in test_images]
deskewed_hogdata_test = np.float32(deskewed_hogdata_test).reshape(-1,81)

# deskewed_hogdata_train
# deskewed_hogdata_test

In [102]:
deskewed_reshape_train = [deskew(im).flatten() for im in train_images]
deskewed_reshape_train = np.float32(deskewed_reshape_train)
deskewed_reshape_train = np.asarray(deskewed_reshape_train)

deskewed_reshape_test = [deskew(im).flatten() for im in test_images]
deskewed_reshape_test = np.float32(deskewed_reshape_test)

# deskewed_reshape_train
# deskewed_reshape_test

In [98]:
deskewed_reshape_train_copy = [deskew(im).flatten() for im in train_images]
deskewed_reshape_shuffle_train = np.float32(deskewed_reshape_train_copy)
np.random.shuffle(deskewed_reshape_shuffle_train)

deskewed_reshape_test = [deskew(im).flatten() for im in test_images]
deskewed_reshape_shuffle_test = np.float32(deskewed_reshape_test)
np.random.shuffle(deskewed_reshape_shuffle_test)

# deskewed_reshape_shuffle_train
# deskewed_reshape_shuffle_test

In [90]:
# helper functions

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from keras import models
from keras import layers
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def grid_search_svc(train_X,train_y,test_X,test_y):
  parameters = [{
    "kernel": ["linear", "poly", "rbf", "sigmoid"], 
    "C": [0.1,1, 10, 100, 1000], 
    "gamma": [0.1, 0.01, 0.001, 0.0001]
  }]
  model = SVC()

  grid_search = GridSearchCV(model, parameters, scoring='accuracy', cv=5)
  grid_search.fit(train_X, train_y)
  clf = grid_search.best_estimator_
  print("Best classifier: ", clf)

  params = grid_search.best_params_
  print("Chosing params: ", params)

  pred_y = clf.predict(test_X)
  print_report(test_y,pred_y)

def grid_search_rfn(train_X,train_y,test_X,test_y):
  parameters = [{
    "max_depth": [10, 15, 20],
    "n_estimators": [50, 100, 120],
    "max_features": [40, 60, 80]
  }]
  model = RandomForestClassifier()

  grid_search = GridSearchCV(model, parameters, scoring='accuracy', cv=5)
  grid_search.fit(train_X, train_y)
  clf = grid_search.best_estimator_
  print("Best classifier: ", clf)

  params = grid_search.best_params_
  print("Chosing params: ", params)

  pred_y = clf.predict(test_X)
  print_report(test_y,pred_y)

def create_model(input_shape):
  model = models.Sequential()
  model.add(layers.Dense(512, activation='relu', input_shape=(input_shape,)))
  model.add(layers.Dense(10, activation='softmax'))
  model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
  return model

def grid_search_nn(train_X,train_y,test_X,test_y,input_shape):
  train_X = train_X.reshape((len(train_X), input_shape))
  test_X = test_X.reshape((len(test_X), input_shape))
  train_X = train_X.astype('float32') / 255
  test_X = test_X.astype('float32') / 255
  encoded_train_y = to_categorical(train_y)
  encoded_test_y = to_categorical(test_y)

  parameters = [{'input_shape':[input_shape],'batch_size': [10, 20, 40, 60, 80, 100], 'epochs': [10, 50, 100] }]
  model = KerasClassifier(build_fn=create_model, verbose=0)

  grid_search = GridSearchCV(model, parameters, n_jobs=-1, cv=5)
  grid_search.fit(train_X, encoded_train_y)
  clf = create_model(input_shape)
  clf.fit(train_X, encoded_train_y, epochs=grid_search.best_params_['epochs'], batch_size=grid_search.best_params_['batch_size'])
  print("Best classifier: ", clf)

  params = grid_search.best_params_
  print("Chosing params: ", params)

  pred_probabilities = clf.predict(test_X)
  pred_y = np.argmax(pred_probabilities,-1)
  print_report(test_y,pred_y)


# Zadanie 2

SVM

In [91]:
# A
grid_search_svc(deskewed_hogdata_train, train_labels, deskewed_hogdata_test, test_labels)

# B
grid_search_svc(hogdata_train, train_labels, hogdata_test, test_labels)

Best classifier:  SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Chosing params:  {'C': 10, 'gamma': 0.1, 'kernel': 'sigmoid'}
97.0
[[ 8  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0]
 [ 0  0  0 11  0  0  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0  0]
 [ 0  0  0  1  0  0  0 13  0  1]
 [ 0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  1 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         8
           3       0.92      1.00      0.96        11
           4       1.00      1.00      1.00        14
          

RandomForest


In [92]:
# A
grid_search_rfn(deskewed_hogdata_train, train_labels, deskewed_hogdata_test, test_labels)

# B
grid_search_rfn(hogdata_train, train_labels, hogdata_test, test_labels)

Best classifier:  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features=40,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=120,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Chosing params:  {'max_depth': 20, 'max_features': 40, 'n_estimators': 120}
96.0
[[ 8  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0]
 [ 0  1  7  0  0  0  0  0  0  0]
 [ 0  0  0 11  0  0  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0  0]
 [ 0  0  0  1  0  0  0 14  0  0]
 [ 0  0  0  0  0  0  0  0  1  1]
 [ 1  0  0  0  0  0  0  0  0 10]]
              precision    re

Neural Network

In [93]:
# A
grid_search_nn(deskewed_hogdata_train, train_labels, deskewed_hogdata_test, test_labels, 81)

# B
grid_search_nn(hogdata_train, train_labels, hogdata_test, test_labels,81)

Epoch 1/100
60/60 [==============================] - 0s 1ms/step - loss: 2.3016 - accuracy: 0.1029
Epoch 2/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2948 - accuracy: 0.1319
Epoch 3/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2857 - accuracy: 0.1229
Epoch 4/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2810 - accuracy: 0.1268
Epoch 5/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2784 - accuracy: 0.1100
Epoch 6/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2613 - accuracy: 0.1327
Epoch 7/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2587 - accuracy: 0.1510
Epoch 8/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2416 - accuracy: 0.1516
Epoch 9/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2025 - accuracy: 0.2395
Epoch 10/100
60/60 [==============================] - 0s 1ms/step - loss: 2.1892 - accuracy: 0.2298
Epoch 11/

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/100
60/60 [==============================] - 1s 1ms/step - loss: 2.3012 - accuracy: 0.1464
Epoch 2/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2942 - accuracy: 0.1220
Epoch 3/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2917 - accuracy: 0.1308
Epoch 4/100
60/60 [==============================] - 0s 2ms/step - loss: 2.2870 - accuracy: 0.1217
Epoch 5/100
60/60 [==============================] - 0s 2ms/step - loss: 2.2804 - accuracy: 0.1391
Epoch 6/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2758 - accuracy: 0.1274
Epoch 7/100
60/60 [==============================] - 0s 2ms/step - loss: 2.2650 - accuracy: 0.1306
Epoch 8/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2454 - accuracy: 0.1435
Epoch 9/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2416 - accuracy: 0.1484
Epoch 10/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2092 - accuracy: 0.2111
Epoch 11/

**Wniosek**: Używanie funkcji "deskew" na obrazach przed zastosowaniem deskryptora HOG zwiększa wynik modelu.

# Zadanie 3

SVM

In [94]:
# A
grid_search_svc(deskewed_hogdata_train, train_labels, deskewed_hogdata_test, test_labels)

# B
grid_search_svc(deskewed_reshape_train, train_labels, deskewed_reshape_test, test_labels)

Best classifier:  SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Chosing params:  {'C': 10, 'gamma': 0.1, 'kernel': 'sigmoid'}
97.0
[[ 8  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0]
 [ 0  0  0 11  0  0  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0  0]
 [ 0  0  0  1  0  0  0 13  0  1]
 [ 0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  1 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         8
           3       0.92      1.00      0.96        11
           4       1.00      1.00      1.00        14
          

RandomForest

In [99]:
# A
grid_search_rfn(deskewed_hogdata_train, train_labels, deskewed_hogdata_test, test_labels)

# B
grid_search_rfn(deskewed_reshape_train, train_labels, deskewed_reshape_test, test_labels)

Best classifier:  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features=40,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Chosing params:  {'max_depth': 15, 'max_features': 40, 'n_estimators': 50}
93.0
[[ 8  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0]
 [ 0  1  7  0  0  0  0  0  0  0]
 [ 0  0  1 10  0  0  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  1  0  9  0  0  0]
 [ 0  0  1  1  0  0  0 13  0  0]
 [ 0  0  0  0  0  0  0  0  1  1]
 [ 1  0  0  0  0  0  0  0  0 10]]
              precision    reca

Neural Network

In [108]:
# A
grid_search_nn(deskewed_hogdata_train, train_labels, deskewed_hogdata_test, test_labels,81)

Epoch 1/100
60/60 [==============================] - 1s 1ms/step - loss: 2.3018 - accuracy: 0.1241
Epoch 2/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2915 - accuracy: 0.1423
Epoch 3/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2897 - accuracy: 0.1356
Epoch 4/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2769 - accuracy: 0.1334
Epoch 5/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2776 - accuracy: 0.1135
Epoch 6/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2674 - accuracy: 0.1248
Epoch 7/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2518 - accuracy: 0.1286
Epoch 8/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2426 - accuracy: 0.1768
Epoch 9/100
60/60 [==============================] - 0s 1ms/step - loss: 2.2252 - accuracy: 0.1873
Epoch 10/100
60/60 [==============================] - 0s 1ms/step - loss: 2.1958 - accuracy: 0.1926
Epoch 11/

In [103]:
# B
grid_search_nn(deskewed_reshape_train, train_labels, deskewed_reshape_test, test_labels, 28*28)

Epoch 1/10
60/60 [==============================] - 1s 6ms/step - loss: 1.1345 - accuracy: 0.6540
Epoch 2/10
60/60 [==============================] - 0s 6ms/step - loss: 0.2459 - accuracy: 0.9408
Epoch 3/10
60/60 [==============================] - 0s 6ms/step - loss: 0.1405 - accuracy: 0.9693
Epoch 4/10
60/60 [==============================] - 0s 6ms/step - loss: 0.0933 - accuracy: 0.9788
Epoch 5/10
60/60 [==============================] - 0s 6ms/step - loss: 0.0369 - accuracy: 0.9957
Epoch 6/10
60/60 [==============================] - 0s 5ms/step - loss: 0.0305 - accuracy: 0.9949
Epoch 7/10
60/60 [==============================] - 0s 5ms/step - loss: 0.0213 - accuracy: 0.9928
Epoch 8/10
60/60 [==============================] - 0s 5ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 9/10
60/60 [==============================] - 0s 6ms/step - loss: 0.0043 - accuracy: 0.9997
Epoch 10/10
60/60 [==============================] - 0s 6ms/step - loss: 0.0040 - accuracy: 1.0000
Best classifier:  <

**Wniosek**: Wyniki klasyfikatorów trenowanych danymi po zadziałaniu na nich funkcją OpenCVHOG uzyskują lepsze rezultaty niż klasyfikatorów trenowanych danymi po wyprostowaniu i spłaszczonymi.

# Zadanie 4

SVM

In [104]:
# A
grid_search_svc(deskewed_reshape_train, train_labels, deskewed_reshape_test, test_labels)

Best classifier:  SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Chosing params:  {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
90.0
[[ 8  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  1  0  0  0]
 [ 0  0  0 10  0  1  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  0  0]
 [ 1  0  0  0  0  5  0  1  0  0]
 [ 0  0  2  0  0  0  8  0  0  0]
 [ 0  0  0  1  0  0  0 13  0  1]
 [ 0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  1  0  0  1  0  9]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       1.00      1.00      1.00        14
           2       0.78      0.88      0.82         8
           3       0.91      0.91      0.91        11
           4       0.93      1.00      0.97        14
          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# B
grid_search_svc(deskewed_reshape_shuffle_train, train_labels, deskewed_reshape_shuffle_test, test_labels)

RandomForest

In [105]:
# A
grid_search_rfn(deskewed_reshape_train, train_labels, deskewed_reshape_test, test_labels)

Best classifier:  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features=40,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=120,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Chosing params:  {'max_depth': 15, 'max_features': 40, 'n_estimators': 120}
88.0
[[ 8  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0]
 [ 0  1  1  9  0  0  0  0  0  0]
 [ 0  0  0  0 13  0  0  0  0  1]
 [ 0  0  1  0  1  5  0  0  0  0]
 [ 1  0  2  0  0  0  7  0  0  0]
 [ 0  0  0  0  0  0  0 14  0  1]
 [ 0  0  1  0  0  0  0  0  1  0]
 [ 0  0  0  0  1  0  0  1  0  9]]
              precision    re

In [ ]:
# B
grid_search_rfn(deskewed_reshape_shuffle_train, train_labels, deskewed_reshape_shuffle_test, test_labels)

Neural Network

In [107]:
# A
grid_search_nn(deskewed_reshape_train, train_labels, deskewed_reshape_test, test_labels, 28*28)

Epoch 1/10
6/6 [==============================] - 0s 9ms/step - loss: 1.6765 - accuracy: 0.4453
Epoch 2/10
6/6 [==============================] - 0s 9ms/step - loss: 0.5280 - accuracy: 0.8983
Epoch 3/10
6/6 [==============================] - 0s 10ms/step - loss: 0.3814 - accuracy: 0.9182
Epoch 4/10
6/6 [==============================] - 0s 9ms/step - loss: 0.2603 - accuracy: 0.9543
Epoch 5/10
6/6 [==============================] - 0s 8ms/step - loss: 0.2018 - accuracy: 0.9543
Epoch 6/10
6/6 [==============================] - 0s 9ms/step - loss: 0.1816 - accuracy: 0.9664
Epoch 7/10
6/6 [==============================] - 0s 9ms/step - loss: 0.1457 - accuracy: 0.9766
Epoch 8/10
6/6 [==============================] - 0s 9ms/step - loss: 0.1036 - accuracy: 0.9776
Epoch 9/10
6/6 [==============================] - 0s 10ms/step - loss: 0.1028 - accuracy: 0.9859
Epoch 10/10
6/6 [==============================] - 0s 9ms/step - loss: 0.0844 - accuracy: 0.9885
Best classifier:  <tensorflow.python.

In [ ]:
# B
grid_search_nn(deskewed_reshape_shuffle_train, train_labels, deskewed_reshape_shuffle_test, test_labels, 28*28)

**Wniosek**: Wyniki dla klasyfikatorów, na których wejściu podaje się dane "zmieszane" są znacznie gorsze niż wyniki dla klasyfikatorów, na których wejściu podaje się dane spłaszczone.